<a href="https://colab.research.google.com/github/deepakboharachhetri/AI_lab/blob/main/Vacuum_cleaner_using_different_agent_type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math
import numpy as np
from collections import deque

GRID_SIZE = 10
MAX_STEPS_SIMPLE = 20

# Fill the matrix using random values
def fill_matrix(char_matrix, row, column):
    for i in range(row):
        for j in range(column):
            ran_value = np.random.rand()
            if ran_value < 0.5:
                char_matrix[i, j] = 'C'
            else:
                char_matrix[i, j] = 'D'

# Create grid using NumPy and fill_matrix
def initialize_grid():
    char_matrix = np.empty((GRID_SIZE, GRID_SIZE), dtype=str)
    fill_matrix(char_matrix, GRID_SIZE, GRID_SIZE)
    grid = char_matrix.tolist()
    dirty_cells = [(i, j) for i in range(GRID_SIZE) for j in range(GRID_SIZE) if grid[i][j] == 'D']
    return grid, dirty_cells

# Print grid
def display_grid(grid, agent_pos):
    for i in range(GRID_SIZE):
        row = ""
        for j in range(GRID_SIZE):
            if (i, j) == agent_pos:
                row += " A "
            elif grid[i][j] == 'D':
                row += " D "
            else:
                row += " . "
        print(row)
    print("-" * 40)

# Get adjacent positions
def get_adjacent(pos):
    x, y = pos
    moves = []
    directions = [
        (-1, 0),  # Up
        (1, 0),   # Down
        (0, -1),  # Left
        (0, 1),   # Right
        (-1, -1), # Up-Left
        (-1, 1),  # Up-Right
        (1, -1),  # Down-Left
        (1, 1),   # Down-Right
    ]
    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < GRID_SIZE and 0 <= new_y < GRID_SIZE:
            moves.append((new_x, new_y))
    return moves


# Simple Reflex Agent
def simple_reflex_agent(grid):
    print("== Simple Reflex Agent ==")
    grid_copy = [row[:] for row in grid]
    agent_pos = (random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1))
    steps = 0
    cleaned = 0

    # Run until no dirty cell remains
    while any(cell == 'D' for row in grid_copy for cell in row):
        steps += 1
        x, y = agent_pos
        print(f"Step {steps}:")
        if grid_copy[x][y] == 'D':
            print(f"Action: SUCK at {agent_pos}")
            grid_copy[x][y] = 'C'
            cleaned += 1
        else:
            adjacent_cells = get_adjacent(agent_pos)
            new_pos = random.choice(adjacent_cells)
            print(f"Action: MOVE from {agent_pos} to {new_pos}")
            agent_pos = new_pos

        display_grid(grid_copy, agent_pos)

    print("All cells are clean. Agent stopped.")
    return steps, cleaned

# Goal-Based Agent
def goal_based_agent(grid):
    print("== Goal-Based Agent ==")
    grid_copy = [row[:] for row in grid]
    agent_pos = (random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1))
    steps = 0
    dirty_positions = {(i, j) for i in range(GRID_SIZE) for j in range(GRID_SIZE) if grid_copy[i][j] == 'D'}

    def bfs(start, goal):
        visited = set()
        queue = deque([(start, [])])
        while queue:
            current, path = queue.popleft()
            if current == goal:
                return path
            if current in visited:
                continue
            visited.add(current)
            for move in get_adjacent(current):
                queue.append((move, path + [move]))
        return []

    while dirty_positions:
        nearest = min(dirty_positions, key=lambda p: abs(p[0] - agent_pos[0]) + abs(p[1] - agent_pos[1]))
        path = bfs(agent_pos, nearest)
        for move in path:
            steps += 1
            print(f"Step {steps}: MOVE to {move}")
            agent_pos = move
            display_grid(grid_copy, agent_pos)
        print(f"Step {steps + 1}: SUCK at {agent_pos}")
        grid_copy[agent_pos[0]][agent_pos[1]] = 'C'
        dirty_positions.remove(agent_pos)
        steps += 1
        display_grid(grid_copy, agent_pos)
    return steps, len(dirty_positions)

# Utility-Based Agent
def utility_based_agent(grid):
    print("== Utility-Based Agent ==")
    grid_copy = [row[:] for row in grid]
    agent_pos = (random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1))
    steps = 0
    utility = 0
    dirty_positions = {(i, j) for i in range(GRID_SIZE) for j in range(GRID_SIZE) if grid_copy[i][j] == 'D'}

    def bfs(start, goal):
        visited = set()
        queue = deque([(start, [])])
        while queue:
            current, path = queue.popleft()
            if current == goal:
                return path
            if current in visited:
                continue
            visited.add(current)
            for move in get_adjacent(current):
                queue.append((move, path + [move]))
        return []

    while dirty_positions:
        nearest = min(dirty_positions, key=lambda p: abs(p[0] - agent_pos[0]) + abs(p[1] - agent_pos[1]))
        path = bfs(agent_pos, nearest)
        for move in path:
            steps += 1
            utility -= 1
            print(f"Step {steps}: MOVE to {move}")
            agent_pos = move
            display_grid(grid_copy, agent_pos)
        print(f"Step {steps + 1}: SUCK at {agent_pos}")
        grid_copy[agent_pos[0]][agent_pos[1]] = 'C'
        dirty_positions.remove(agent_pos)
        utility += 5
        steps += 1
        display_grid(grid_copy, agent_pos)
    return steps, utility

# Run all agents and compare
def run_agents():
    grid, _ = initialize_grid()
    print("Initial Grid:")
    display_grid(grid, (-1, -1))  # show without agent

    s_steps, s_cleaned = simple_reflex_agent(grid)
    g_steps, _ = goal_based_agent(grid)
    u_steps, u_utility = utility_based_agent(grid)

    print("\n=== COMPARISON ===")
    print(f"Simple Reflex Agent: Steps = {s_steps}, Cleaned = {s_cleaned}")
    print(f"Goal-Based Agent   : Steps = {g_steps}, Cleaned = All")
    print(f"Utility-Based Agent: Steps = {u_steps}, Utility = {u_utility}")

run_agents()


Initial Grid:
 .  .  D  D  D  .  .  .  D  D 
 .  .  .  .  D  D  .  .  .  . 
 D  D  D  D  .  D  .  D  .  D 
 D  .  .  .  .  .  .  D  D  D 
 D  .  D  .  .  .  .  D  .  D 
 D  .  D  .  .  D  .  D  D  D 
 .  D  D  D  .  .  .  .  D  D 
 .  .  .  D  .  D  D  .  D  . 
 .  .  .  .  D  D  .  .  .  D 
 .  .  D  .  D  D  .  .  D  D 
----------------------------------------
== Simple Reflex Agent ==
Step 1:
Action: SUCK at (5, 0)
 .  .  D  D  D  .  .  .  D  D 
 .  .  .  .  D  D  .  .  .  . 
 D  D  D  D  .  D  .  D  .  D 
 D  .  .  .  .  .  .  D  D  D 
 D  .  D  .  .  .  .  D  .  D 
 A  .  D  .  .  D  .  D  D  D 
 .  D  D  D  .  .  .  .  D  D 
 .  .  .  D  .  D  D  .  D  . 
 .  .  .  .  D  D  .  .  .  D 
 .  .  D  .  D  D  .  .  D  D 
----------------------------------------
Step 2:
Action: MOVE from (5, 0) to (4, 1)
 .  .  D  D  D  .  .  .  D  D 
 .  .  .  .  D  D  .  .  .  . 
 D  D  D  D  .  D  .  D  .  D 
 D  .  .  .  .  .  .  D  D  D 
 D  A  D  .  .  .  .  D  .  D 
 .  .  D  .  .  D  .  D  D  D